In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = "/content/drive/MyDrive/AudioPatternRecognition/Final_Project/Data.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists (Dictionary Data in the JSON file) to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

In [3]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split => (from sklearn.model_selection import train_test_split)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis (extra dimention) to input sets (CNN expects 3 dimention data suvh as (130,13,1))
    X_train = X_train[..., np.newaxis]  #... => means what ever the X_train is, + new axis => The result would be 4-dimention: (NumberOfSamples,130,13,1)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [4]:
def build_model(input_shape):   #By (import tensorflow.keras as keras) => the input_shape is the 3-dimentions of training set
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    # Input layer: a visual representation of a song, where the x-axis represents time, the y-axis represents frequency, and the color intensity represents the amplitude of the frequencies.
    model = keras.Sequential()

    # 1st conv layer
    # analyzing the spectrogram to detect basic patterns, like the rhythm of a drum (for rock) or the smooth flow of a saxophone (for jazz).
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))  #32 KERNELS(fILTERS), GRID SIZE (3*3) ==> Also for the first layer we should identify the input shape

    # This is like zooming out of the spectrogram to focus on the most important features (e.g., the overall rhythm or dominant frequencies) while ignoring finer details.
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))

    # 2nd conv layer => Exactly the same as previous layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))

    # 3rd conv layer => Filter size and Max pool size is changed.
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    # flatten output (which is 2-dimentions) into 1-dimension and feed it into dense layer
    model.add(keras.layers.Flatten()) #Imagine taking all the features you’ve extracted (e.g., rhythms, harmonies, frequencies) and listing them in a single, long list for further analysis.

    #add a dense layer with 64 neurons (fully connected layer for classification) ==> This is like analyzing the list of features to determine which combinations are most important for classifying the music genre (e.g., "rock music often has strong beats and electric guitar riffs").
    model.add(keras.layers.Dense(64, activation='relu'))

    # output layer ==> This is the final step where the model predicts the genre of the song (e.g., "80% rock, 10% pop, 10% jazz").
    model.add(keras.layers.Dense(10, activation='softmax')) #As we have 10 genre => we need 10 output neurons each specifies 1 genre

    return model


In [5]:
def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1) => we insert new axis in the begining of the array.

    # perform prediction => a 2 dimantional array and have 10 different values as scores for our 10 different genres: [[0.1, 0.2, ...]] ==> we should extract the index with the max value.
    prediction = model.predict(X)   # X -> (1,130,13,1) => the forst dimention (1) is the number of samples we want to predict (usually usefull in batch prediction)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1) # argmax => extract the index with the max value => [4]

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

In [6]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs

        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 11, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 6, 32)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 62, 4, 32)           │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 31, 2, 32)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 30, 1, 32)           │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 15, 1, 32)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 480)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          30,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,130 (176.29 KB)

 Trainable params: 45,130 (176.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 13s 56ms/step - accuracy: 0.1888 - loss: 3.7925 - val_accuracy: 0.3138 - val_loss: 1.9518
Epoch 2/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.3388 - loss: 1.8673 - val_accuracy: 0.3852 - val_loss: 1.6992
Epoch 3/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.4216 - loss: 1.6024 - val_accuracy: 0.4252 - val_loss: 1.6200
Epoch 4/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.4750 - loss: 1.4522 - val_accuracy: 0.4326 - val_loss: 1.5325
Epoch 5/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.5015 - loss: 1.4045 - val_accuracy: 0.4673 - val_loss: 1.4632
Epoch 6/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.5169 - loss: 1.3335 - val_accuracy: 0.4826 - val_loss: 1.4247
Epoch 7/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.5367 - loss: 1.2835 - val_accuracy: 0.4820 - val_loss: 1.3832
Epoch 8/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - accuracy: 0.5428 - loss: 1.2471 - va

79/79 - 1s - 10ms/step - accuracy: 0.6460 - loss: 1.0060

Test accuracy: 0.6459751725196838
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


Target: 2, Predicted label: [5]


In [7]:
if __name__ == "__main__":

    #1. get train, validation, test splits => (Train the model with train dataset, use validation dataset for model optimization, test the model with test dataset which model hasnt seen befor)
    # 0.25: This is the fraction of the dataset to be allocated for testing. For example, if the dataset contains 1,000 samples, 25% (250 samples) will be used as the test set.
    # 0.2: This is the fraction of the remaining data (after splitting off the test set) to be allocated for validation. For example, if 750 samples remain after the test split, 20% (150 samples) will be used as the validation set.
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    #2. create network (CNN Model)
    input_shape = (X_train.shape[1], X_train.shape[2], 1)   #As explaines in the "prepare_datasets" function, the X_train data set are 4-dimentional which weneed the 3 dimentions
    model = build_model(input_shape)

    #3. compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    #4. train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    # plot accuracy/error for training and validation
    plot_history(history)

    #5. evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    # pick a sample to predict from the test set
    X_to_predict = X_test[100]
    y_to_predict = y_test[100]

    #6. predict sample
    predict(model, X_to_predict, y_to_predict)
